<a href="https://colab.research.google.com/github/ShuHuiK/WIE3007_Group_Assignment/blob/ShuHui/USE_THIS_Step_1_Dataset_Simulation_%26_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1 - Dataset Simulation & Feature Engineering**

In [2]:
# Step 1: Dataset Simulation with ALL 20 Columns for Top 10 Rows
print("="*100)
print("STEP 1: DATASET SIMULATION & FEATURE ENGINEERING")
print("="*100)

!pip install faker
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random

# Initialize
fake = Faker()
np.random.seed(42)

def generate_realistic_financial_data(n_records=1000):
    """Generate realistic financial dataset with correlations"""
    print(f"\n📊 Generating {n_records} financial records...")

    data = []
    sectors = ['Technology', 'Healthcare', 'Finance', 'Retail', 'Manufacturing', 'Services', 'Real Estate', 'Education']

    for i in range(n_records):
        # Basic demographics
        age = np.random.normal(45, 15)
        age = max(18, min(70, int(age)))

        # Correlated features
        if age < 30:
            income = np.random.lognormal(10.5, 0.4)  # Lower for young
            employment_years = np.random.exponential(2)
        elif age < 50:
            income = np.random.lognormal(11.2, 0.3)  # Peak earning
            employment_years = np.random.exponential(8)
        else:
            income = np.random.lognormal(10.8, 0.35)  # Slightly lower for older
            employment_years = np.random.exponential(15)

        income = max(20000, min(300000, income))

        # Credit score with some correlation to income
        base_credit = np.random.normal(650, 100)
        income_effect = (income - 75000) / 10000  # $10k = 10 points
        credit_score = int(max(300, min(850, base_credit + income_effect)))

        # Loan amount based on income and credit score
        loan_multiplier = 0.5 + (credit_score / 850) * 2
        loan_amount = np.random.exponential(income * loan_multiplier)
        loan_amount = max(1000, min(500000, loan_amount))

        # Debt-to-income ratio (correlated with credit score)
        if credit_score > 700:
            dti = np.random.beta(2, 8) * 0.4  # Low DTI for good credit
        else:
            dti = np.random.beta(4, 4) * 0.8  # Higher DTI for poor credit

        # Generate realistic feedback
        feedback_options = [
            "Excellent service! The loan process was smooth and efficient.",
            "Very satisfied with the quick approval and competitive rates.",
            "High interest rates compared to competitors.",
            "Faced multiple delays in loan processing.",
            "Professional staff and transparent fee structure.",
            "Hidden fees that weren't explained clearly upfront.",
            "Great online banking platform, easy to use and reliable.",
            "Customer support was helpful and resolved my issue promptly.",
            "Process took longer than expected but got approved eventually.",
            "Average experience, similar to other banks I've used.",
            "Quick loan disbursement, appreciated the efficiency.",
            "Had issues with the mobile banking app interface.",
            "Very responsive customer service team.",
            "Fees are reasonable for the services provided.",
            "Application process was confusing and lengthy.",
            "Online account management is user-friendly.",
            "Interest rates could be more competitive.",
            "Paperwork requirements were excessive.",
            "Fast approval process, less than 48 hours.",
            "Communication from the bank was unclear at times."
        ]
        feedback = np.random.choice(feedback_options)

        # Transaction patterns
        if income > 100000:
            transaction_freq = np.random.poisson(45)
            avg_transaction = np.random.lognormal(6, 0.5)
        else:
            transaction_freq = np.random.poisson(25)
            avg_transaction = np.random.lognormal(5, 0.4)

        # Business characteristics
        sector = np.random.choice(sectors)
        if sector in ['Technology', 'Finance']:
            company_size = np.random.choice(['Medium', 'Large'], p=[0.4, 0.6])
        else:
            company_size = np.random.choice(['Small', 'Medium', 'Large'], p=[0.5, 0.3, 0.2])

        # Transaction description based on sector
        transaction_descs = {
            'Technology': ["Software subscription payment", "Cloud services invoice", "Hardware equipment purchase", "Tech support services"],
            'Finance': ["Stock trading commission", "Investment management fee", "Banking service charges", "Financial software license"],
            'Healthcare': ["Medical equipment purchase", "Health insurance premium", "Pharmacy supplies", "Medical software subscription"],
            'Retail': ["Inventory restocking", "Store rental payment", "Marketing campaign cost", "POS system maintenance"],
            'Manufacturing': ["Raw materials purchase", "Equipment maintenance", "Factory utilities", "Shipping logistics"],
            'Services': ["Consulting fee payment", "Service contract renewal", "Professional development", "Office supplies"],
            'Real Estate': ["Property maintenance", "Mortgage payment", "Real estate taxes", "Insurance premium"],
            'Education': ["Tuition fee payment", "Educational materials", "Campus facilities", "Library resources"],
            'General': ["Monthly salary deposit", "Grocery shopping", "Utility bill payment", "Credit card payment", "Fuel purchase", "Restaurant dining"]
        }

        if sector in transaction_descs:
            transaction_desc = np.random.choice(transaction_descs[sector])
        else:
            transaction_desc = np.random.choice(transaction_descs['General'])

        # Default probability (target variable)
        default_prob = 1 / (1 + np.exp(-(
            -2.5 +
            (850 - credit_score) * 0.01 +
            dti * 3 +
            (loan_amount / max(income, 1)) * 0.5 -
            employment_years * 0.1
        )))

        default = 1 if np.random.random() < default_prob else 0

        # Risk level
        if default_prob > 0.3:
            risk_level = 'High'
        elif default_prob > 0.1:
            risk_level = 'Medium'
        else:
            risk_level = 'Low'

        record = {
            'customer_id': f"CUST{10000 + i:05d}",
            'age': age,
            'income': round(income, 2),
            'credit_score': credit_score,
            'account_balance': round(np.random.exponential(5000), 2),
            'loan_amount': round(loan_amount, 2),
            'loan_duration': np.random.choice([12, 24, 36, 60, 84]),
            'employment_years': round(min(employment_years, 40), 1),
            'debt_to_income': round(dti, 3),
            'transaction_frequency': transaction_freq,
            'avg_transaction': round(avg_transaction, 2),
            'business_sector': sector,
            'company_size': company_size,
            'customer_feedback': feedback[:100] + "..." if len(feedback) > 100 else feedback,
            'transaction_description': transaction_desc,
            'location': f"{fake.city()}, {fake.state_abbr()}",
            'last_interaction_days': int(np.random.exponential(30)),
            'num_products': np.random.poisson(2) + 1,
            'default_history': default,
            'risk_level': risk_level
        }

        data.append(record)

    df = pd.DataFrame(data)
    print(f"✅ Generated {len(df)} records with {len(df.columns)} features")
    return df

# Generate the dataset
df = generate_realistic_financial_data(1000)

# Display ALL 20 columns for top 10 rows
print("\n" + "="*100)
print("TOP 10 ROWS - SHOWING ALL 20 COLUMNS")
print("="*100)

# Configure pandas display to show ALL columns without truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect terminal width
pd.set_option('display.max_colwidth', 40)  # Maximum column width
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines
pd.set_option('display.float_format', '{:,.2f}'.format)  # Format floats

# Get the top 10 rows
top_10 = df.head(10).copy()

# Format specific columns for better readability
def format_currency(x):
    return f"${x:,.2f}"

def format_percent(x):
    return f"{x*100:.1f}%"

# Apply formatting to specific columns
top_10_display = top_10.copy()
top_10_display['income'] = top_10_display['income'].apply(lambda x: f"${x:,.0f}")
top_10_display['account_balance'] = top_10_display['account_balance'].apply(lambda x: f"${x:,.2f}")
top_10_display['loan_amount'] = top_10_display['loan_amount'].apply(lambda x: f"${x:,.0f}")
top_10_display['debt_to_income'] = top_10_display['debt_to_income'].apply(lambda x: f"{x*100:.1f}%")
top_10_display['avg_transaction'] = top_10_display['avg_transaction'].apply(lambda x: f"${x:,.2f}")

print("\n📋 Showing all 20 columns for first 10 records:")
print("-" * 180)

# Create a nicely formatted table
print(top_10_display.to_string(index=False,
                               formatters={
                                   'age': lambda x: f"{x:3d}",
                                   'credit_score': lambda x: f"{x:3d}",
                                   'loan_duration': lambda x: f"{x:3d} months",
                                   'employment_years': lambda x: f"{x:4.1f} years",
                                   'transaction_frequency': lambda x: f"{x:3d}/month",
                                   'last_interaction_days': lambda x: f"{x:3d} days",
                                   'num_products': lambda x: f"{x:1d}",
                                   'default_history': lambda x: "Yes" if x == 1 else "No"
                               }))

print("-" * 180)

# Show column information
print("\n📝 COLUMN INFORMATION (20 columns total):")
print("-" * 80)

columns_info = [
    ("customer_id", "object", "Unique customer identifier"),
    ("age", "int64", "Customer age in years"),
    ("income", "float64", "Annual income in USD"),
    ("credit_score", "int64", "FICO credit score (300-850)"),
    ("account_balance", "float64", "Current bank account balance"),
    ("loan_amount", "float64", "Requested loan amount"),
    ("loan_duration", "int64", "Loan term in months"),
    ("employment_years", "float64", "Years at current employment"),
    ("debt_to_income", "float64", "Debt to income ratio"),
    ("transaction_frequency", "int64", "Monthly transaction count"),
    ("avg_transaction", "float64", "Average transaction amount"),
    ("business_sector", "object", "Industry sector of business"),
    ("company_size", "object", "Size classification (Small/Medium/Large)"),
    ("customer_feedback", "object", "Customer service feedback text"),
    ("transaction_description", "object", "Description of recent transaction"),
    ("location", "object", "City and state"),
    ("last_interaction_days", "int64", "Days since last interaction"),
    ("num_products", "int64", "Number of financial products held"),
    ("default_history", "int64", "Target: 1=Defaulted, 0=No default"),
    ("risk_level", "object", "Risk category (High/Medium/Low)")
]

for i, (col_name, col_type, col_desc) in enumerate(columns_info, 1):
    print(f"{i:2d}. {col_name:25} ({col_type:10}) - {col_desc}")

print("-" * 80)

# Show dataset statistics
print("\n📊 DATASET STATISTICS:")
print(f"Total records: {df.shape[0]:,}")
print(f"Total columns: {df.shape[1]}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n📈 NUMERICAL FEATURES SUMMARY:")
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('default_history')  # Remove target from summary

for col in numeric_cols[:5]:  # Show first 5 numeric columns
    print(f"  {col:25}: Mean = ${df[col].mean():,.0f}" if 'income' in col or 'loan' in col or 'balance' in col else
          f"  {col:25}: Mean = {df[col].mean():.1f}")

print("\n🎯 TARGET VARIABLE ANALYSIS:")
default_counts = df['default_history'].value_counts()
print(f"  No Default (0): {default_counts.get(0, 0):,} records ({default_counts.get(0, 0)/len(df)*100:.1f}%)")
print(f"  Defaulted (1):   {default_counts.get(1, 0):,} records ({default_counts.get(1, 0)/len(df)*100:.1f}%)")

print("\n📊 CATEGORICAL FEATURES DISTRIBUTION:")
categorical_cols = ['business_sector', 'company_size', 'risk_level']
for col in categorical_cols:
    print(f"\n  {col}:")
    value_counts = df[col].value_counts().head(3)
    for value, count in value_counts.items():
        print(f"    • {value:15}: {count:3d} records ({count/len(df)*100:.1f}%)")

print("\n" + "="*100)
print("DATASET SIMULATION COMPLETE - READY FOR FEATURE ENGINEERING")
print("="*100)

STEP 1: DATASET SIMULATION & FEATURE ENGINEERING
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00

📊 Generating 1000 financial records...
✅ Generated 1000 records with 20 features

TOP 10 ROWS - SHOWING ALL 20 COLUMNS

📋 Showing all 20 columns for first 10 records:
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
customer_id age  income credit_score account_balance loan_amount loan_duration employment_years debt_to_income transaction_frequency avg_transaction business_sector company_size                                              customer_feedback   transaction_description               location last_interaction_days num_products default_history risk_level
  CUST10000  52 $46,705          535       $2,552.97     $13,932     60 months       19.8 years          40.4%              27/month         $222.34   Manufacturing        S